In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [4]:
flights_initial = pd.read_excel("flights.xlsx")

In [22]:
flights = flights_initial.copy()

In [23]:
flights.head()

day_of_week    fl_date airline  flight_number tail_number origin_airport  \
0            6  2015-8-29      BA            542      N951AT            ABE   
1            3  2015-8-19      BA            542      N981AT            ABE   
2            4  2015-8-13      BA           1801      N925AT            ABE   
3            6  2015-8-22      BA            542      N998AT            ABE   
4            4  2015-8-27      BA            542      N965AT            ABE   

  origin_state_code destination_airport destination_state_code  \
0                PA                 ATL                     GA   
1                PA                 ATL                     GA   
2                PA                 ATL                     GA   
3                PA                 ATL                     GA   
4                PA                 ATL                     GA   

   departure_time  ...  taxi_in  arrival_time  arrival_delay  diverted  \
0           644.0  ...      6.0         844.0            0.0         0   
1           623.0  ...      6.0         814.0            1.0         0   
2           615.0  ...      7.0         806.0           -9.0         0   
3           641.0  ...      8.0         843.0           -1.0         0   
4           614.0  ...     10.0         823.0           10.0         0   

   cancelled  nas_delay  security_delay  airline_delay  late_aircraft_delay  \
0          0        NaN             NaN            NaN                  NaN   
1          0        NaN             NaN            NaN                  NaN   
2          0        NaN             NaN            NaN                  NaN   
3          0        NaN             NaN            NaN                  NaN   
4          0        NaN             NaN            NaN                  NaN   

   weather_delay  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  

[5 rows x 24 columns]

In [24]:
flights.columns

Index(['day_of_week', 'fl_date', 'airline', 'flight_number', 'tail_number',
       'origin_airport', 'origin_state_code', 'destination_airport',
       'destination_state_code', 'departure_time', 'departure_delay',
       'taxi_out', 'air_time', 'distance', 'taxi_in', 'arrival_time',
       'arrival_delay', 'diverted', 'cancelled', 'nas_delay', 'security_delay',
       'airline_delay', 'late_aircraft_delay', 'weather_delay'],
      dtype='object')

In [38]:
#Create cols for flight delay
flights['arrival_delay_type'] = np.nan

#Give different statuses according to length of delay or cancelled or diverted
flights.loc[flights['arrival_delay'] <= 0.0, 'arrival_delay_type'] = 'ON-TIME'
flights.loc[flights['arrival_delay'] > 0.0, 'arrival_delay_type'] = 'MINOR'
flights.loc[flights['arrival_delay'] >= 15, 'arrival_delay_type'] = 'SIGNIFICANT'
flights.loc[flights['arrival_delay'] >= 45, 'arrival_delay_type'] = 'SEVERE'
flights.loc[flights['cancelled'] == 1, 'arrival_delay_type'] = 'CANCELLED'
flights.loc[flights['diverted'] == 1, 'arrival_delay_type'] = 'DIVERTED'

In [39]:
flights['arrival_delay_type'].unique()

array(['ON-TIME', 'MINOR', 'SIGNIFICANT', 'SEVERE', 'CANCELLED',
       'DIVERTED'], dtype=object)

In [40]:
#If empty, means our filtering is good
flights[flights['arrival_delay_type'].isnull()]

Empty DataFrame
Columns: [day_of_week, fl_date, airline, flight_number, tail_number, origin_airport, origin_state_code, destination_airport, destination_state_code, departure_time, departure_delay, taxi_out, air_time, distance, taxi_in, arrival_time, arrival_delay, diverted, cancelled, nas_delay, security_delay, airline_delay, late_aircraft_delay, weather_delay, arrival_delay_type]
Index: []

[0 rows x 25 columns]

# Part 1 - Overall flight profile
Comparing the delayed vs on-time

In [66]:
delay_type_count_table = flights.groupby(['airline', 'arrival_delay_type'])['flight_number'].aggregate(['count'])
delay_type_count_table

count
airline arrival_delay_type       
AA      CANCELLED             790
        DIVERTED              250
        MINOR               12454
        ON-TIME             51806
        SEVERE               6157
        SIGNIFICANT          8291
BA      CANCELLED              69
        DIVERTED              203
        MINOR               11986
        ON-TIME             57197
        SEVERE               4798
        SIGNIFICANT          6694

In [67]:
AA_delayed = delay_type_count_table.loc(axis=0)['AA', ['MINOR', 'SEVERE', 'SIGNIFICANT']]
BA_delayed = delay_type_count_table.loc(axis=0)['BA', ['MINOR', 'SEVERE', 'SIGNIFICANT']]
AA_ontime = delay_type_count_table.loc(axis=0)['AA', 'ON-TIME']
BA_ontime = delay_type_count_table.loc(axis=0)['BA', 'ON-TIME']
AA_delayed['count'].sum()

26902

In [68]:
total_AA = AA_ontime['count'].sum() + AA_delayed['count'].sum()
total_BA = BA_ontime['count'].sum() + BA_delayed['count'].sum()
pd.DataFrame({
    'ON-TIME': np.array([AA_ontime['count'].sum() / total_AA, BA_ontime['count'].sum() / total_BA]),
    'DELAYED': np.array([AA_delayed['count'].sum() / total_AA, BA_delayed['count'].sum() / total_BA])
}, index=['AA', 'BA'])

ON-TIME   DELAYED
AA  0.658205  0.341795
BA  0.708980  0.291020

# Part 2 - Average delay over days